In [ ]:
using RigidBodyDynamics, RigidBodyDynamics.Contact, RigidBodyDynamics.OdeIntegrators
using MomentumBasedControl
using MixedIntegerExperiments
using StaticArrays
using RigidBodyTreeInspector, DrakeVisualizer
using ValkyrieRobot
using BipedControlUtil

In [ ]:
val = Valkyrie()
mechanism = val.mechanism;

In [ ]:
# add environment
rootframe = root_frame(mechanism)
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
leftwall = HalfSpace3D(Point3D(rootframe, 0., -1., 1.5), FreeVector3D(rootframe, 0., 1., 0.))
rightwall = HalfSpace3D(Point3D(rootframe, 0, 1., 1.5), FreeVector3D(rootframe, 0., -1., 0.))
add_environment_primitive!(mechanism, floor)
# add_environment_primitive!(mechanism, leftwall)
# add_environment_primitive!(mechanism, rightwall)
;

In [ ]:
# Create visualizer
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1))
vis = Visualizer(parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]), :valkyrie);
visualize_environment(mechanism, vis)

In [ ]:
remove_fixed_tree_joints!(mechanism)
const state = MechanismState(Float64, mechanism)

In [ ]:
# Initial setup
zero!(state)
kneebend = 1.
for side in instances(Side)
    set_configuration!(state, val.knees[side], [kneebend])
    set_configuration!(state, val.hippitches[side], [-kneebend / 2])
    set_configuration!(state, val.anklepitches[side], [-kneebend / 2])
end
set_configuration!(state, val.floatingjoint, [1; 0; 0; 0; 0; 0; 1.08]) # TODO
settransform!(vis, state);

In [ ]:
const τ = Vector{Float64}(num_velocities(mechanism))
const result = DynamicsResult(Float64, mechanism)
function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    τ[:] = velocity(state)
    scale!(τ, -1.0)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

In [ ]:
const controller = MomentumBasedController{Float64}(mechanism);
contacts = add_mechanism_contacts!(controller)
jointacceltasks = add_mechanism_joint_accel_tasks!(controller);

In [ ]:
comgains = PDGains(10., 2.)

In [ ]:
footacceltasks = Dict{Side, SpatialAccelerationTask{Float64}}()
for side in instances(Side)
    foot = val.feet[side]
    p = path(mechanism, root_body(mechanism), foot)
    frame = default_frame(foot)
    task = footacceltasks[side] = SpatialAccelerationTask(p, frame, eye(3, 3), eye(3, 3))
    add!(controller, task)
end

In [ ]:
const result = DynamicsResult(Float64, mechanism)
feet = val.feet
com0 = center_of_mass(state)
μ = 0.8
normal = floor.outward_normal
function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(controller)
    MomentumBasedControl.reset!(controller)
    regularize_joint_accels!(controller, 0.05)
    for side in instances(Side)
        task = footacceltasks[side]
        base = RigidBodyDynamics.Graphs.source(task.path)
        body = RigidBodyDynamics.Graphs.target(task.path)
        accel = zero(SpatialAcceleration{T}, default_frame(body), default_frame(base), default_frame(body))
        set!(task, accel, Inf)
        
        for contactsettings in contacts[body]
            set!(contactsettings, 1e-1, μ, transform(state, normal, default_frame(body)))
        end
    end
    
    # Centroidal momentum control
    com = center_of_mass(state)
    centroidal = centroidal_frame(controller)
    world_to_centroidal = Transform3D(com.frame, centroidal, -com.v)
    h = transform(momentum(state), world_to_centroidal)
    comerror = FreeVector3D(transform(com0, world_to_centroidal))
    comderror = -FreeVector3D(h.frame, h.linear)
    ldot_desired = comgains.k * comerror + comgains.d * comderror
    hdot_desired = Wrench(ldot_desired, zero(ldot_desired))
    add!(controller, MomentumRateTask(hdot_desired, eye(SMatrix{3, 3}), eye(SMatrix{3, 3}), 1.))
    
    τ = control(controller, t, state)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

In [ ]:
# Simulation
const dyn! = controlled_dynamics!
# const dyn! = damped_dynamics!
const integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), DrakeVisualizerSink(vis));

In [ ]:
integrate(integrator, state, 1., 5e-4, maxRealtimeRate = Inf)